In [12]:
# Below is a copy of slurm_train.py from anurag/slurm_onebutton

import torchvision

# import some common libraries
import numpy as np
import sys
if '/opt/ros/kinetic/lib/python2.7/dist-packages' in sys.path:
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')

import cv2
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import random
import os
import numpy as np
import json

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.config import CfgNode as CN
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data import DatasetMapper, build_detection_train_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader, build_detection_train_loader
import detectron2.data.transforms as T
import shutil
from setuptools.namespaces import flatten

import random
import torch 
import base64
import io
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from ast import literal_eval
from PIL import Image


coco_yaml = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
lvis_yaml = "LVIS-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"
lvis_yaml2 = "LVIS-InstanceSegmentation/mask_rcnn_R_101_FPN_1x.yaml"
pano_yaml = "COCO-PanopticSegmentation/panoptic_fpn_R_50_3x.yaml"

jsons_root = '/checkpoint/apratik/finals/jsons/active_vision/'
img_dir_test = '/checkpoint/apratik/ActiveVision/active_vision/replica_random_exploration_data/frl_apartment_1/rgb'
test_jsons = ['frlapt1_20n0.json', 'frlapt1_20n1.json', 'frlapt1_20n2.json']
test_jsons = [os.path.join(jsons_root, x) for x in test_jsons]

val_json = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200/83/seg/coco_train.json'
img_dir_val = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200/83/rgb'

## Detectron2 Setup

# from copy_paste import CopyPaste
# import albumentations as A

class Trainer(DefaultTrainer):
#     @classmethod
#     def build_evaluator(cls, cfg, dataset_name, output_folder=None):
#         if output_folder is None:
#             output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
#         return COCOEvaluator(dataset_name, output_dir=output_folder)
    
    @classmethod
    def build_train_loader(cls, cfg):
        mapper = DatasetMapper(cfg, is_train=True, augmentations=[
            T.ResizeShortestEdge(short_edge_length=cfg.INPUT.MIN_SIZE_TRAIN, max_size=1333, sample_style='choice'),
            T.RandomFlip(prob=0.5),
            T.RandomCrop("absolute", (640, 640)),
            T.RandomBrightness(0.9, 1.1)
        ])
        return build_detection_train_loader(cfg, mapper=mapper)

class COCOTrain:
    def __init__(self, lr, w, maxiters, seed):
        self.cfg = get_cfg()
        self.cfg.merge_from_file(model_zoo.get_config_file(coco_yaml))
        self.cfg.SOLVER.BASE_LR = lr  # pick a good LR
        self.cfg.SOLVER.MAX_ITER = maxiters
        self.cfg.SOLVER.WARMUP_ITERS = w
        self.seed = seed
        
    def reset(self, train_json, img_dir_train, dataset_name):
        DatasetCatalog.clear()
        MetadataCatalog.clear()
        self.train_data = dataset_name +  "_train"
        self.dataset_name = dataset_name
        self.train_json = train_json
        register_coco_instances(self.train_data, {}, train_json, img_dir_train)
        self.results = {
            "bbox": {
                "AP50": []
            },
            "segm": {
                "AP50": []
            }
        }
        self.val_results = {
            "bbox": {
                "AP50": []
            },
            "segm": {
                "AP50": []
            }
        }
    
    def vis(self):
        dataset_dicts = DatasetCatalog.get(self.train_data)
        for d in random.sample(dataset_dicts, 2):
            img = cv2.imread(d["file_name"])
            visualizer = Visualizer(img[:, :, ::-1], metadata=MetadataCatalog.get(self.train_data), scale=0.5)
            vis = visualizer.draw_dataset_dict(d)
            img = vis.get_image()
            plt.figure(figsize=(12,8))
            plt.imshow(img)
            plt.show()
            
    def train(self):
        cfg = self.cfg
        print(f'SOLVER PARAMS {cfg.SOLVER.MAX_ITER, cfg.SOLVER.WARMUP_ITERS, cfg.SOLVER.BASE_LR}')
        cfg.DATASETS.TRAIN = (self.train_data,)
        cfg.DATASETS.TEST = ()
        cfg.DATALOADER.NUM_WORKERS = 2
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(coco_yaml)  # Let training initialize from model zoo
        cfg.SOLVER.IMS_PER_BATCH = 2
        cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128 
        MetadataCatalog.get(self.train_data).thing_classes = ['chair', 'cushion', 'door', 'indoor-plant', 'sofa', 'table']
        print(f'classes {MetadataCatalog.get(self.train_data)}')
        cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(MetadataCatalog.get(self.train_data).get("thing_classes"))  
        cfg.OUTPUT_DIR = os.path.join('output_aug', str(cfg.SOLVER.MAX_ITER), self.dataset_name + str(self.seed))
        print(f"recreating {cfg.OUTPUT_DIR}")
        # if os.path.isdir(cfg.OUTPUT_DIR):
        #     shutil.rmtree(cfg.OUTPUT_DIR)
        print(cfg.OUTPUT_DIR)
        os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
        self.trainer = Trainer(cfg) #DefaultTrainer(cfg) 
        self.trainer.resume_or_load(resume=False)
        self.trainer.train()
        
    def run_val(self, dataset_name, val_json, img_dir_val):
        self.val_data = dataset_name + "_val" + str(self.seed)
        self.val_json = val_json
        self.cfg.DATASETS.TEST = (self.val_data,)
        register_coco_instances(self.val_data, {}, val_json, img_dir_val)
        MetadataCatalog.get(self.val_data).thing_classes = ['chair', 'cushion', 'door', 'indoor-plant', 'sofa', 'table']
        print(f'classes {MetadataCatalog.get(self.val_data)}')
        self.evaluator = COCOEvaluator(self.val_data, ("bbox", "segm"), False, output_dir=self.cfg.OUTPUT_DIR)
        self.val_loader = build_detection_test_loader(self.cfg, self.val_data)
        results = inference_on_dataset(self.trainer.model, self.val_loader, self.evaluator)
        self.val_results['bbox']['AP50'].append(results['bbox']['AP50'])
        self.val_results['segm']['AP50'].append(results['segm']['AP50'])
        return results

    def run_test(self, dataset_name, test_json, img_dir_test):
        self.test_data = dataset_name + "_test" + str(self.seed)
        self.test_json = test_json
        self.cfg.DATASETS.TEST = (self.test_data,)
        register_coco_instances(self.test_data, {}, test_json, img_dir_test)
        MetadataCatalog.get(self.test_data).thing_classes = ['chair', 'cushion', 'door', 'indoor-plant', 'sofa', 'table']
        print(f'classes {MetadataCatalog.get(self.test_data)}')
        self.evaluator = COCOEvaluator(self.test_data, ("bbox", "segm"), False, output_dir=self.cfg.OUTPUT_DIR)
        self.val_loader = build_detection_test_loader(self.cfg, self.test_data)
        results = inference_on_dataset(self.trainer.model, self.val_loader, self.evaluator)
        self.results['bbox']['AP50'].append(results['bbox']['AP50'])
        self.results['segm']['AP50'].append(results['segm']['AP50'])
        return results
        
    def run_train(self, train_json, img_dir_train, dataset_name):
        self.reset(train_json, img_dir_train, dataset_name)
        # self.vis()
        self.train()


maxiters = [500, 800]
lrs = [0.0001, 0.0005, 0.001, 0.002, 0.005]
warmups = [100, 200]

def write_summary_to_file(filename, results, header_str):
    if isinstance(results['bbox']['AP50'][0], list):
        results['bbox']['AP50'] = list(flatten(results['bbox']['AP50']))
        results['segm']['AP50'] = list(flatten(results['segm']['AP50']))
    with open(filename, "a") as f:
        f.write(header_str)
        f.write(f"\nbbox AP50 {sum(results['bbox']['AP50'])/len(results['bbox']['AP50'])}")
        f.write(f"\nsegm AP50 {sum(results['segm']['AP50'])/len(results['segm']['AP50'])}")
        f.write(f'\nall results {results}')

from pathlib import Path
import string

def run_training(out_dir, img_dir_train, n, traj, x, gt, p):
    results_dir = os.path.join('results1021', str(traj), x, str(gt), str(p))
    if not os.path.isdir(results_dir):
        os.makedirs(results_dir)
    train_json = os.path.join(out_dir, 'coco_train.json')
    for lr in lrs:
        for warmup in warmups:
            for maxiter in maxiters:
                results = {
                    "bbox": {
                        "AP50": []
                    },
                    "segm": {
                        "AP50": []
                    }
                }
                for i in range(n):
                    c = COCOTrain(lr, warmup, maxiter, i)
                    dataset_name = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(7))
                    print(f'dataset_name {dataset_name}')
                    c.run_train(train_json, img_dir_train, dataset_name)
                    res_eval = c.run_val(dataset_name, val_json, img_dir_val)
                    with open(os.path.join(results_dir, "validation_results.txt"), "a") as f:
                        f.write(f'lr {lr} warmup {warmup} maxiter {maxiter}\n')
                        f.write(json.dumps(res_eval) + '\n')
                    for yix in range(len(test_jsons)):
                        r = c.run_test(dataset_name + str(yix), test_jsons[yix], img_dir_test)
                        with open(os.path.join(results_dir, "all_results.txt"), "a") as f:
                            f.write(json.dumps(r) + '\n')
                    print(f'all results {c.results}')
                    results['bbox']['AP50'].append(c.results['bbox']['AP50'])
                    results['segm']['AP50'].append(c.results['segm']['AP50'])
                    write_summary_to_file(os.path.join(results_dir, str(n) + '_granular.txt'), c.results, f'\ntrain_json {train_json}')
                
                itername = str(lr) + ' ' + str(warmup) + ' ' + str(maxiter) + ' ' + str(n)
                write_summary_to_file(os.path.join(results_dir, itername + '_results_averaged.txt'), results, f'\ntrain_json {train_json}, average over {n} runs')
   

In [13]:
data_path = '/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019'
job_folder = '/checkpoint/apratik/jobs/active_vision/pipeline/apartment_0/straightline/no_noise/14-10-2021/19:17:17'

import glob

def _runner(trajs, gt, ps):
    for traj in trajs:
        for p in ps:
            for x in ['default', 'activeonly']:
                traj_path = os.path.join(data_path, str(traj), x)
                if os.path.isdir(traj_path):
                    outdir = os.path.join(job_folder, str(traj), x, f'pred_label_gt{gt}p{p}')
                    if len(glob.glob1(outdir,"*.npy")) > 0:
                        run_training(outdir, os.path.join(traj_path, 'rgb'), 1, traj, x, gt, p)
    
            

In [14]:
_runner([1,3,6,10,13], 5, [2,4,6,8])

dataset_name MEWZYRX
SOLVER PARAMS (500, 100, 0.0001)
classes Metadata(evaluator_type='coco', image_root='/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/rgb', json_file='/checkpoint/apratik/jobs/active_vision/pipeline/apartment_0/straightline/no_noise/14-10-2021/19:17:17/1/default/pred_label_gt5p2/coco_train.json', name='MEWZYRX_train', thing_classes=['chair', 'cushion', 'door', 'indoor-plant', 'sofa', 'table'])
recreating output_aug/500/MEWZYRX0
output_aug/500/MEWZYRX0
[10/21 06:01:03 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (7, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (7,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (24, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (24,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (6, 256, 1, 1) 

[10/21 06:01:04 d2.engine.train_loop]: Starting training from iteration 0
[10/21 06:01:10 d2.utils.events]:  eta: 0:02:32  iter: 19  total_loss: 3.571  loss_cls: 2.023  loss_box_reg: 0.6218  loss_mask: 0.6919  loss_rpn_cls: 0.1404  loss_rpn_loc: 0.03771  time: 0.3204  data_time: 0.0153  lr: 1.9081e-05  max_mem: 3290M
[10/21 06:01:16 d2.utils.events]:  eta: 0:02:23  iter: 39  total_loss: 2.985  loss_cls: 1.419  loss_box_reg: 0.6147  loss_mask: 0.6838  loss_rpn_cls: 0.06408  loss_rpn_loc: 0.02979  time: 0.3097  data_time: 0.0036  lr: 3.9061e-05  max_mem: 3290M
[10/21 06:01:22 d2.utils.events]:  eta: 0:02:14  iter: 59  total_loss: 2.276  loss_cls: 0.7749  loss_box_reg: 0.6721  loss_mask: 0.6727  loss_rpn_cls: 0.05928  loss_rpn_loc: 0.03121  time: 0.2995  data_time: 0.0036  lr: 5.9041e-05  max_mem: 3290M
[10/21 06:01:28 d2.utils.events]:  eta: 0:02:07  iter: 79  total_loss: 1.974  loss_cls: 0.6056  loss_box_reg: 0.6369  loss_mask: 0.652  loss_rpn_cls: 0.03666  loss_rpn_loc: 0.03185  time: 

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (7, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (7,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (24, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (24,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (6, 256, 1, 1) 

[10/21 06:06:53 d2.engine.train_loop]: Starting training from iteration 0
[10/21 06:06:59 d2.utils.events]:  eta: 0:03:30  iter: 19  total_loss: 3.35  loss_cls: 1.851  loss_box_reg: 0.6252  loss_mask: 0.6932  loss_rpn_cls: 0.1089  loss_rpn_loc: 0.03007  time: 0.2766  data_time: 0.0166  lr: 1.9081e-05  max_mem: 3290M
[10/21 06:07:05 d2.utils.events]:  eta: 0:03:30  iter: 39  total_loss: 2.944  loss_cls: 1.33  loss_box_reg: 0.6396  loss_mask: 0.6863  loss_rpn_cls: 0.1265  loss_rpn_loc: 0.03939  time: 0.2794  data_time: 0.0035  lr: 3.9061e-05  max_mem: 3290M
[10/21 06:07:11 d2.utils.events]:  eta: 0:03:25  iter: 59  total_loss: 2.316  loss_cls: 0.8117  loss_box_reg: 0.6722  loss_mask: 0.6726  loss_rpn_cls: 0.04793  loss_rpn_loc: 0.02973  time: 0.2787  data_time: 0.0036  lr: 5.9041e-05  max_mem: 3290M
[10/21 06:07:16 d2.utils.events]:  eta: 0:03:19  iter: 79  total_loss: 2.036  loss_cls: 0.6672  loss_box_reg: 0.6687  loss_mask: 0.6522  loss_rpn_cls: 0.01931  loss_rpn_loc: 0.03366  time: 0.

...

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (7, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (7,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (24, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (24,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (6, 256, 1, 1) 

[10/21 10:07:28 d2.engine.train_loop]: Starting training from iteration 0
[10/21 10:07:33 d2.utils.events]:  eta: 0:01:38  iter: 19  total_loss: 3.353  loss_cls: 1.494  loss_box_reg: 0.8729  loss_mask: 0.6847  loss_rpn_cls: 0.1375  loss_rpn_loc: 0.0997  time: 0.2064  data_time: 0.0213  lr: 0.00047952  max_mem: 3290M
[10/21 10:07:37 d2.utils.events]:  eta: 0:01:35  iter: 39  total_loss: 2.475  loss_cls: 0.7312  loss_box_reg: 0.8795  loss_mask: 0.6152  loss_rpn_cls: 0.09707  loss_rpn_loc: 0.1743  time: 0.2058  data_time: 0.0040  lr: 0.00097902  max_mem: 3290M
[10/21 10:07:41 d2.utils.events]:  eta: 0:01:31  iter: 59  total_loss: 2.027  loss_cls: 0.5508  loss_box_reg: 0.8532  loss_mask: 0.4905  loss_rpn_cls: 0.02798  loss_rpn_loc: 0.1044  time: 0.2077  data_time: 0.0039  lr: 0.0014785  max_mem: 3290M
[10/21 10:07:45 d2.utils.events]:  eta: 0:01:27  iter: 79  total_loss: 1.478  loss_cls: 0.2663  loss_box_reg: 0.7352  loss_mask: 0.3597  loss_rpn_cls: 0.01993  loss_rpn_loc: 0.09537  time: 0.

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (7, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (7,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (24, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (24,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (6, 256, 1, 1) 

[10/21 10:10:39 d2.engine.train_loop]: Starting training from iteration 0
[10/21 10:10:43 d2.utils.events]:  eta: 0:02:35  iter: 19  total_loss: 3.287  loss_cls: 1.52  loss_box_reg: 0.8921  loss_mask: 0.687  loss_rpn_cls: 0.1241  loss_rpn_loc: 0.1632  time: 0.2003  data_time: 0.0147  lr: 0.00047953  max_mem: 3290M
[10/21 10:10:47 d2.utils.events]:  eta: 0:02:32  iter: 39  total_loss: 2.43  loss_cls: 0.7724  loss_box_reg: 0.8843  loss_mask: 0.6231  loss_rpn_cls: 0.07181  loss_rpn_loc: 0.1096  time: 0.2013  data_time: 0.0034  lr: 0.00097903  max_mem: 3290M
[10/21 10:10:51 d2.utils.events]:  eta: 0:02:27  iter: 59  total_loss: 2.151  loss_cls: 0.5464  loss_box_reg: 0.8306  loss_mask: 0.5325  loss_rpn_cls: 0.03796  loss_rpn_loc: 0.1419  time: 0.2009  data_time: 0.0033  lr: 0.0014785  max_mem: 3290M
[10/21 10:10:55 d2.utils.events]:  eta: 0:02:23  iter: 79  total_loss: 1.533  loss_cls: 0.3462  loss_box_reg: 0.7467  loss_mask: 0.3418  loss_rpn_cls: 0.0172  loss_rpn_loc: 0.08276  time: 0.2006

FileNotFoundError: [Errno 2] No such file or directory: '/checkpoint/apratik/jobs/active_vision/pipeline/apartment_0/straightline/no_noise/14-10-2021/19:17:17/1/default/pred_label_gt5p4/coco_train.json'